### I. Why Cython Works
#### Interpreted Versus Compiled Execution
#### Dynamic Verses Static Typing
Cython可以提升性能的主要原因是其将<font color='#ff2222'>**静态类型**</font>引入了动态语言。

### II. 使用**cdef**声明静态类型
##### 1. cdef关键字+类型
##### 2. cdef: + 代码块

In [1]:
%load_ext Cython

In [2]:
%%cython
def integrate(a,b,f):
    cdef int i
    cdef int N = 2000
    cdef float dx, s=0.0
    '''
    cdef:
        int i
        int N = 2000
        float dx, s=0.0
    '''
    dx = (b-a)/N
    for i in range(N):
        s += f(a+i*dx)
    return s*dx

Cython不能使用static关键字声明变量，但是可以使用const关键字，这将在第7和8章讲解。
**Cython supports the full range of C declarations**
<font color=#ff22aa size=2 face="微软雅黑">Table 3-1 Various cdef declarations: page 54</font>
- Pointer指针
    - cdef int *p
    - cdef void **buf
- Stack-allocated C arrays
    - cdef int arr[10]
    - cdef double points[20][20]
- typedefed aliased types???
    - cdef size_t len
- Compound types (struts and unions)
    - cdef tm time_struct
    - cdef int_short_union_t hi_lo_bytes
- Function pointers
    - cdef void (*f)(int, double)


### III. Automatic Type Inference in Cython
cdef并不是Cython中声明静态变量的唯一方法。Cython对函数或方法主体中未声明类型的变量进行变量类型推断。在默认的情况下，只有当变量类型推断不会改变代码语法的情况下，Cython才会这么做。
- <font color='#ff2222'>@cython.infer_types(True)</font>
- 当我们启动infer_types时，我么要自行保证整形操作不会溢出导致语法（相对于untyped version）改变。

In [3]:
%%cython
cimport cython
@cython.infer_types(True)
def more_inference():
    i = 1
    d = 2.0
    c = 3 + 4j
    r = i * d + c
    return r

### IV. C Pointers in Cython
<font color='#ff22aa' size=3>学习C语言语法</font>

### V. Mixing Statically and Dynamically Typed Variables
Cython允许静态和动态类型变量之间相互赋值。
<font color=#ff22aa size=2 face="微软雅黑">Table 3-2 Type correspondence between built-in Python types and C or C++ types: page 58</font>
- bint
    - 在C中，bint布尔整形是一个int，可以与Python的bool相互转化。0为False，1为True。
- 整形的转化和溢出
    - 在Pyton2中int被保存成C long，Python long则具有无限的精度；而在Python3中所有的整形都具有无限的精度。
    - 当把Python的整形转化为C整形时，Cython会生成检查溢出的代码，如果C类型不能表示Python整形，a runtime OverflowError is raised.
- 浮点型的转化
    - Python float被保存成C double。将一个Python float转化为C float会将其截为0.0或正/负无穷。
- 复数类型
    - Python的复数类型存储为两个C double组成的C struct。
    - Cython有float complex和double complex类型（C-level types），对应于Python complex，并具有和Python complex相同的接口，但使用更高效的C级别操作。
        - real，imag属性
        - conjugate方法
        - 高效的加减乘除运算
- byte type
    - Python的byte类型和char *或std::string自动转化
- str and unicode types
    - 需要设定<font color='#ff2222'>c_string_type和c_string_encoding编译指令</font>，才能允许str或unicode类型和char *或std::string相互转化。


### VI. 使用Python类型静态声明变量Statically Declaring Variables with a Python Type
对于Python的内建类型<font face='consolas'>list, tuple, list</font>，扩展类型<font face='consolas'>NumPy，及很多其他类型</font>，我们可以使用cdef静态的声明变量。这种静态声明要求<1>Python类型必须由C实现，<2>Cython必须由access to the declaration.

In [4]:
%%cython
# cdef声明python内建类型
cdef list particles, modified_particles
cdef dict names_from_particles
cdef str pname
cdef set unique_particles
'''
particles = list(names_from_particles.keys())
# 动态变量可以从静态声明的Python类型中初始化
other_particles = particles
# 删除other_particles[0]也会同时删除particles[0]
del other_particles[0]
# other_particles可以指向其他任何Python类型
# 而particles只能指向Python的list类型
'''

当我们加减乘除标量时，如果操作对象时是动态Python对象类型，以Python语法执行操作；如果操作对象是静态C变量类型，则执行的是C语法。这里有一个特例，就是无论操作对象时Python还是C类型，Cython执行除法和求余时默认使用Python语法。要使用C语法，则需要使用<font face='consolas'>cdivision</font>编译指令。
- at global module level, or in a directive comment
> \# cython: cdivision=True
- at function level with a decorator
- within a function with a context manager


In [5]:
%%cython
# at function level with a decorator
cimport cython
@cython.cdivision(False)
def divides(int a, int b):
    return a / b

# within a function with a context manager
cimport cython 
def remainder(int a, int b):
    with cython.cdivision(True):
        return a % b

### VII. Static Typing for Speed
提供的静态类型信息越多，Cython优化的效果越好。除了少部分的情况外，这通常是正确的。

### VIII. Reference Counting and Static String Types
Python的一个主要特征是自动内存管理。Cython通过直接的引用计数来实现它，自动垃圾回收器会周期性的清理unreachable reference cycles.
应用类型的计算需引入中间变量，若将计算结果直接赋给另一个引用变量，中间产生的临时变量会被垃圾回收机制回收而导致问题。

In [ ]:
# 以下是错误的,b1+b2的临时结果会被垃圾回收机制回收
b1 = b"All men are mortal."
b2 = b"Socrates is a man."
cdef char *buf = b1 + b2
# 正确的方式是引入中间变量tmp,tmp不会被回收
tmp = s1 + s2
cdef char *buf = tmp
# or
cdef bytes tmp = s1 + s2
cdef char *buf = tmp

### IX Cython's Three Kinds of Functions
- Python函数
    - 可以在导入时构建或在运行时动态构建
    - 可以通过lambda关键字匿名构建
    - 可以在其他函数或其他nested scope中定义
    - 可以由其他函数返回
    - 可以作为参数传入其他函数
    - 可以按位置或参数名传入参数
    - 可以定义参数的默认值
- C函数
    - 可以作为参数传入其他函数，但是这么做远比Python中难
    - 不可以在其他函数中定义
    - 有一个不可修改的statically assigned name
    - 只能按位置传入参数
    - 不支持默认参数值
    
#### [1] Python Functions in Cython with the <font color='#ff2222'>def</font> Keyword
Cython支持使用def关键字定义Python function，其函数类型是builtin_function_or_method；而Python下定义的相同函数类型为function，且其具有可变的名字。由于Cython版本运行的是编译的C代码，而Python通过解释器执行bytecodes，所以Cython版本要比Python版本快。

In [6]:
%%file fact.pyx
def py_fact(n):
    '''Compute n!'''
    if n <= 1:
        return 1
    return n * py_fact(n-1)

Overwriting fact.pyx


In [7]:
import pyximport
pyximport.install()
import fact
type(fact.py_fact)

builtin_function_or_method

In [8]:
# Python version
def interpreted_fact(n):
    '''Compute n!'''
    if n <= 1:
        return 1
    return n * interpreted_fact(n-1)
type(interpreted_fact)

function

In [9]:
%timeit fact.py_fact(20)
%timeit interpreted_fact(20)

The slowest run took 6.05 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 1.39 µs per loop
100000 loops, best of 3: 2.71 µs per loop


#### [2] C Function in Cython with the <font color='#ff2222'>cdef</font> keyword
当被用来定义函数时，cdef关键字通过C-calling sementics创建函数，其参数及返回来行都是典型的静态类型，他们可以work with C指针对象、structs以及其他不能自动装换成Python类型的C类型。可以将cdef定义的函数当做通过Cython的类Python语法定义的C函数。

我们可以在cdef函数中声明和使用Python对象和动态变量，或接受它们作为参数。但由于我们通常希望通过cdef获得尽可能接近于C的表现所以通常这种做法并不好。

Cython允许在一个Cython源码中定义cdef和def函数。cdef的返回类型可以是任何我们见过的静态类型，包括指针、structs、C数组以、静态Python类型（譬如list、dict）和void。如果省略返回类型，则默认类型为object。

cdef定义的函数可以被同一源码中的其他def或cdef函数调用，但是如果要在其他外部Python代码中调用cdef函数，需要将其包装为def函数。

In [10]:
%%cython
cdef long c_fact(long n):
    '''Computes n!'''
    if n <= 1:
        return 1
    return n * c_fact(n-1)
def wrap_c_fact(n):
    '''Compute n!'''
    return c_fact(n)

In [11]:
%timeit wrap_c_fact(20)

The slowest run took 18.45 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 79 ns per loop


#### [3] Combining def and cdef Functions with <font color='#ff2222'>cpdef</font>
通过cpdef函数我们可以同时得到C版本的函数以及它的Python wrapper。在Cython中调用cpdef函数时，我们只调用C-only version，而在Python中调用它时，则会调用wrapper。

cpdef有一个限制：cdef需要兼容Python和Ctypes。任何Python对象可以以C表示，但是并不是所有的C类型都可以以Python来表示；所以我们不能任意的使用void、C指针或C数组作为cpdef函数的参数或返回类型。(参见<font color='#ff2222'>Table 3-2</font>)

<font color='ff22aa' face='微软雅黑'>inline关键字????</font>

In [13]:
%%cython
cpdef long cp_fact(long n):
    '''Compute n!'''
    if n <= 1:
        return 1
    return n * cp_fact(n-1)

In [14]:
%timeit cp_fact(20)

The slowest run took 19.65 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 74.3 ns per loop


#### [4] Functions and Exception Handling
def函数总是在C级别返回某种类型的PyObject指针，这种不变形使得Cython可以正确的传递def函数产生的exceptions。但Cython的cdef以及cpdef函数类型可能返回的是非Python类型，这就需要其他exception指示机制。Cython通过except来实现。
> <font color='#ff2222' face='consols'>except? -1</font>

- -1可以是其他任意返回类型取值范围内的整数，用来作为sentinel指示exeption发生了。
- 由于-1也可能是函数的返回值，所以需要使用?，使得在这种情况下不发送exception状态。如果返回的exeption indicator不存在这种模糊性，则可以省略?。
- 我们也可以使用<font color='#ff2222' face='consols'>except *</font>，使得Cython无论返回何值都检查expection是否生成，但是这样会增加一些开销。

#### [5] Functions and the embedsignature Compiler Directive
将<font face='consolas'>embedsignature</font>设为True，可以将Cython编译的函数签名添加到docstring输出中。(<font color='#ff2222'>page 46</font>)
> <font face='consolas'># cython: embedsignature=True</font>

### X. Type Coercion and Casting
<font color='#ff2222'>page 74</font>

Cyhton的casting operator和C的飞翔相似，只是其用尖括号代替了圆括号。
> Cython: cdef int \*pre_i = &lt;int*&gt; v

> C: int \*ptr_i = (int*)v;

C中的显示类型转换不会被检查，要让Cython在转化之前进行类型检查，可以使用checked casting operator。
> cdef list cast_list = &lt;list<font color='#ff2222'>?</font>&gt;a

### XI. Declaring and Using structs, unions and enums
Cython可以使用cdef或ctypedef关键字定义struct及union类型。其有三种初始化方法：
- use struct literals
- 对struct的field分别赋值
- 使用Python字典进行赋值，这会增加一定的开销

Cython不支持嵌套及匿名的struct和union。要使用嵌套struct和union，需要将内部的struct进行定义并为其提供dummy names。 

In [17]:
%%cython
'''
# 对应的C代码
struct mycpx {
    int a;
    float b
}
union uu {
    int a;
    short b, c;
}
'''
# cdef也可以换成ctypedef
cdef struct mycpx:
    float real
    float imag
cdef union uu:
    int a
    short b,c
# 声明具有mycpx类型的变量
cdef mycpx zz

'''struct的初始化'''
# <1> 
cdef mycpx a = mycpx(3.1415, -1.0)
cdef mycpx b = mycpx(real=2.718, imag=1.618034)
# <2>
zz.real = 3.1415
zz.imag = -1.0
# <3>
cdef mycpx zz2 = {'real':3.1415, 'imag': -1.0}

'''nested C struct declaration'''
cdef struct _inner:
    int inner_a
cdef struct nested:
    int outer_a
    _inner inner
# 初始化和赋值
cdef nested n = {'outer_a': 1, 'inner': {'inner_a': 2}}

'''enum的定义,可以使用cdef/ctypedef，可以匿名'''
cdef enum PRIMARIES:
    RED = 1
    YELLOW = 3
    BLUE = 5
cdef enum SECONDARIES:
    ORANGE, GREEN, PUPPLE

### XII. Type Aliasing with ctypedef
Cython可以通过ctypedef关键字定义类型别名，这和C中的typedef相似，在和使用typedef别名的外部代码交互时很重要。
ctypedef语句只能在file scope出现，不能再函数内部或其他局部声明local type name。

<font color='#ff22aa'>Fused Types and Generic Programming——page 78</font>

fused type是Cython中新的类型，其提供三个内建fused types：<font color='#ff2222' face='consolas'>integral, floating, numeric</font>。
- integral: short, int, long (C)
- floating: float, double (C)
- numeric: integral, floating, float complex, double complex

In [19]:
%%cython
# 只需改变下面ctypedef一行代码就可以切换计算的精度。
ctypedef double real
ctypedef long integral
def displacement(real d0, real v0, real a, real t):
    """Calculates displacement under constant acceleration."""
    cdef real d = d0 + (v0 * t) + (0.5 * a * t**2)
    return d

# fused type
from cython cimport integral
cpdef integral integral_max(integral a, integral b):
    return a if a >= b else b

cimport cython
ctypedef fused integral_or_floating:
    cython.short
    cython.int
    cython.long
    cython.float
    cython.double
cpdef integral_or_floating generic_max(integral_or_floating a,
                                       integral_or_floating b):
    return a if a >= b else b

### XIII. Cython for Loops and while Loops
#### [1] Guidelines for Efficient Loops
- When looping over a range over a <font color='#ff2222' face='consolas'>range</font> call, we should type the <font color='#ff2222' face='consolas'>range</font> argument as a C integer。如果我们确信index value不会导致整形溢出，我们也需要静态定义其类型。
- When looping over a container(<font color='#ff2222' face='consolas'>list, tuple, dict, etc</font>),静态定义loop indexing variable的类型，可能引入更多的开销。此时为了更高效的执行循环，可以考虑将container转化为<font color='#ff2222' face='consolas'>C++ equivalent containe</font>或者使用<font color='#ff2222' face='consolas'>typed memoryviews</font>。
- 为了保证高效的使用while循环，我们需要使循环条件表达式足够高效。这可能涉及到使用typed variables和cdef functions。

In [34]:
%%cython
# 定义for循环中的i和n静态类型，提升循环效率
cpdef long sum_c(long n):
    cdef long i, sum = 0
    for i in range(n):
        sum += i
    return sum

def sum_cpy(n):
    sum = 0
    for i in range(n):
        sum += i
    return sum

In [35]:
def sum_py(n):
    sum = 0
    for i in range(n):
        sum += i
    return sum

In [37]:
'''
Cython cpdef循环、Cython def循环、Python循环
的效率比较。
'''
%timeit sum_c(1000)
%timeit sum_cpy(1000)
%timeit sum_py(1000)

1000000 loops, best of 3: 510 ns per loop
100000 loops, best of 3: 19.6 µs per loop
10000 loops, best of 3: 34.5 µs per loop


### XIIII. The Cython Preprocessor——page 81
- Cython使用DEF关键字构建宏——C语言的宏参考http://c.biancheng.net/cpp/html/1552.html

- IF-ELIF-ELSE compile-time statement

### XV. Bridging the Python 2 and Python 3 Divide——page 82